# Running the strategy

In [1]:
! python /home/jovyan/work/q_pack/q_run/run_BT.py \
--strat_name=simple_strategy_2 \
--mode=backtest \
--tickers=EUR_USD,GBP_USD \
--ml_log=True

Strategy run finished with Run ID: 4
ML Log Saved in Minio Bucket: model-support-files as 4_ml_log.csv
Profit ... or Loss: 32.62


In [21]:
! python /home/jovyan/work/q_pack/q_run/run_BT.py \
--strat_name=simple_strategy_2 \
--mode=backtest \
--tickers=EUR_USD,GBP_USD,AUD_USD,EUR_JPY,NZD_USD \
--ml_log=True

Strategy run finished with Run ID: 19
{'EUR_USD'}
security  Length= 2421
datetime  Length= 2421
close  Length= 2421
RSI  Length= 2421
STOCHASTIC  Length= 2421
ATR  Length= 2421
1
{'GBP_USD'}
security  Length= 2419
datetime  Length= 2419
close  Length= 2419
RSI  Length= 2419
STOCHASTIC  Length= 2419
ATR  Length= 2419
2
{'AUD_USD'}
security  Length= 2420
datetime  Length= 2420
close  Length= 2420
RSI  Length= 2420
STOCHASTIC  Length= 2420
ATR  Length= 2420
3
{'EUR_JPY'}
security  Length= 2419
datetime  Length= 2419
close  Length= 2419
RSI  Length= 2419
STOCHASTIC  Length= 2419
ATR  Length= 2419
4
{'NZD_USD'}
security  Length= 2423
datetime  Length= 2423
close  Length= 2423
RSI  Length= 2423
STOCHASTIC  Length= 2423
ATR  Length= 2423
5
ML Log Saved in Minio Bucket: model-support-files as 19_ml_log.csv
Profit ... or Loss: 79.46


In [ ]:
! python /home/jovyan/work/q_pack/q_run/run_BT.py \
--strat_name=simple_strategy_1 \
--mode=backtest \
--tickers=EUR_USD,GBP_USD,AUD_USD,EUR_JPY,NZD_USD \
--ml_log=True

In [16]:
import os
import warnings
import sys
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import mlflow
import mlflow.sklearn
import mlflow.pyfunc
import io
import boto3

# Preprocessing the file

In [17]:
s3 = boto3.client('s3',endpoint_url="http://minio-image:9000",aws_access_key_id="minio-image",aws_secret_access_key="minio-image-pass")
data = pd.read_csv(s3.get_object(Bucket="model-support-files", Key='18_ml_log.csv')['Body'],sep=',')
data.head(10)

,security,datetime,close,RSI,STOCHASTIC,ATR,security.1,datetime.1,close.1,RSI.1,...,close.3,RSI.3,STOCHASTIC.3,ATR.3,security.4,datetime.4,close.4,RSI.4,STOCHASTIC.4,ATR.4
0,EUR_USD,2010-12-30 22:00:00,1.32855,NaN,NaN,NaN,GBP_USD,2010-12-30 22:00:00,1.54250,NaN,...,108.294,NaN,NaN,NaN,NZD_USD,2010-12-30 22:00:00,0.77039,NaN,NaN,NaN
1,EUR_USD,2010-12-31 22:00:00,1.33815,NaN,NaN,NaN,GBP_USD,2011-01-01 22:00:00,1.55359,NaN,...,108.478,NaN,NaN,NaN,NZD_USD,2010-12-31 22:00:00,0.77988,NaN,NaN,NaN
2,EUR_USD,2011-01-01 22:00:00,1.33280,NaN,NaN,NaN,GBP_USD,2011-01-02 22:00:00,1.54343,NaN,...,108.335,NaN,NaN,NaN,NZD_USD,2011-01-01 22:00:00,0.77481,NaN,NaN,NaN
3,EUR_USD,2011-01-02 22:00:00,1.32512,NaN,NaN,NaN,GBP_USD,2011-01-03 22:00:00,1.54550,NaN,...,107.800,NaN,NaN,NaN,NZD_USD,2011-01-02 22:00:00,0.77285,NaN,NaN,NaN
4,EUR_USD,2011-01-03 22:00:00,1.32919,NaN,NaN,NaN,GBP_USD,2011-01-04 22:00:00,1.54506,NaN,...,108.627,NaN,NaN,NaN,NZD_USD,2011-01-03 22:00:00,0.76397,NaN,NaN,NaN
5,EUR_USD,2011-01-04 22:00:00,1.31258,NaN,NaN,0.007796,GBP_USD,2011-01-05 22:00:00,1.54488,NaN,...,108.517,NaN,NaN,0.966800,NZD_USD,2011-01-04 22:00:00,0.75626,NaN,NaN,0.003766
6,EUR_USD,2011-01-05 22:00:00,1.29978,NaN,NaN,0.007135,GBP_USD,2011-01-06 22:00:00,1.54070,NaN,...,108.114,NaN,NaN,1.002840,NZD_USD,2011-01-05 22:00:00,0.75544,NaN,NaN,0.003841
7,EUR_USD,2011-01-06 22:00:00,1.29057,NaN,NaN,0.006176,GBP_USD,2011-01-08 22:00:00,1.55004,NaN,...,106.948,NaN,NaN,0.890272,NZD_USD,2011-01-06 22:00:00,0.75512,NaN,NaN,0.004753
8,EUR_USD,2011-01-08 22:00:00,1.28607,NaN,NaN,0.005193,GBP_USD,2011-01-09 22:00:00,1.54752,NaN,...,107.000,NaN,NaN,0.818618,NZD_USD,2011-01-07 22:00:00,0.76040,NaN,NaN,0.004858
9,EUR_USD,2011-01-09 22:00:00,1.28742,NaN,NaN,0.006228,GBP_USD,2011-01-10 22:00:00,1.55133,NaN,...,106.826,NaN,NaN,0.789894,NZD_USD,2011-01-08 22:00:00,0.75680,NaN,NaN,0.004334


In [18]:
from ml_pack.preprocessing.ml_preprocessing import ml_preprocessing
preprocessed_file=ml_preprocessing(input_file="18_ml_log.csv",fwd_returns=5)
preprocessed_file

'processed_18_ml_log.csv'

In [19]:
s3 = boto3.client('s3',endpoint_url="http://minio-image:9000",aws_access_key_id="minio-image",aws_secret_access_key="minio-image-pass")
data = pd.read_csv(s3.get_object(Bucket="model-support-files", Key=preprocessed_file)['Body'],sep=',')

In [20]:
data.head(10)

,RSI,STOCHASTIC,security.1,datetime.1,close.1,RSI.1,STOCHASTIC.1,ATR.1,security.2,datetime.2,...,RSI.3,STOCHASTIC.3,ATR.3,security.4,datetime.4,close.4,RSI.4,STOCHASTIC.4,ATR.4,fwd_returns
0,56.991619,78.882076,GBP_USD,2011-02-01 22:00:00,1.61286,67.632318,67.385341,0.017257,AUD_USD,2011-01-31 22:00:00,...,60.432136,61.626107,0.935730,NZD_USD,2011-01-30 22:00:00,0.76849,49.073894,60.403681,0.006121,-0.005146
1,59.448606,77.279931,GBP_USD,2011-02-02 22:00:00,1.61216,67.387740,72.292758,0.016800,AUD_USD,2011-02-01 22:00:00,...,61.668242,58.322696,1.005984,NZD_USD,2011-01-31 22:00:00,0.77122,50.438378,54.132662,0.007706,0.003864
2,60.945457,79.322084,GBP_USD,2011-02-03 22:00:00,1.60366,64.459591,63.829216,0.014456,AUD_USD,2011-02-02 22:00:00,...,62.520782,56.724459,1.012587,NZD_USD,2011-02-01 22:00:00,0.77171,50.683722,54.940994,0.008413,0.013553
3,56.546196,72.484186,GBP_USD,2011-02-04 22:00:00,1.61105,65.796292,58.935306,0.013043,AUD_USD,2011-02-03 22:00:00,...,57.820031,47.065073,0.942670,NZD_USD,2011-02-02 22:00:00,0.76904,49.307831,53.562590,0.007401,-0.000088
4,54.832174,60.957090,GBP_USD,2011-02-05 22:00:00,1.60738,64.549062,55.468523,0.010434,AUD_USD,2011-02-04 22:00:00,...,56.768554,34.925276,0.958136,NZD_USD,2011-02-03 22:00:00,0.76670,48.123427,47.674873,0.007064,-0.002328
5,55.613371,49.955854,GBP_USD,2011-02-06 22:00:00,1.60899,64.851430,59.334204,0.010579,AUD_USD,2011-02-05 22:00:00,...,59.101824,30.068226,0.938109,NZD_USD,2011-02-04 22:00:00,0.77001,49.884947,45.460466,0.006314,0.000744
6,54.703221,46.218141,GBP_USD,2011-02-07 22:00:00,1.60283,62.733622,53.372498,0.009913,AUD_USD,2011-02-06 22:00:00,...,58.423712,33.211501,0.803887,NZD_USD,2011-02-05 22:00:00,0.76661,48.147683,42.295167,0.005323,-0.010511
7,53.706989,40.152433,GBP_USD,2011-02-08 22:00:00,1.60332,62.833497,50.427908,0.009863,AUD_USD,2011-02-07 22:00:00,...,57.668422,33.184575,0.795709,NZD_USD,2011-02-06 22:00:00,0.76784,48.814782,43.780122,0.005452,-0.018827
8,55.140150,31.906370,GBP_USD,2011-02-09 22:00:00,1.60121,62.092209,44.785321,0.009994,AUD_USD,2011-02-08 22:00:00,...,58.689905,29.736497,0.850768,NZD_USD,2011-02-07 22:00:00,0.76966,49.803305,43.324215,0.006538,-0.002733
9,55.991085,29.008934,GBP_USD,2011-02-10 22:00:00,1.59634,60.391076,40.078329,0.010017,AUD_USD,2011-02-09 22:00:00,...,60.000598,26.764242,1.002014,NZD_USD,2011-02-08 22:00:00,0.77020,50.099104,48.000521,0.006430,0.004991


# Train a ML-model using MLflow

### Point to the MLflow tracking endpoint

In [13]:
mlflow.tracking.set_tracking_uri('http://mlflow-image:5500')

### Create a MLflow experiment

In [15]:
# mlflow.create_experiment(name='simple_trading_ml_models', artifact_location='s3://mlflow-models')

In [16]:
mlflow.set_experiment('simple_trading_ml_models')

### Fetching the preprocessed data from Minio

In [17]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

warnings.filterwarnings("ignore")
np.random.seed(40)
# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "fwd_returns"
train_x = train.drop(["fwd_returns"], axis=1)
test_x = test.drop(["fwd_returns"], axis=1)
train_y = train[["fwd_returns"]]
test_y = test[["fwd_returns"]]

n_estimators=200
max_depth=10

lr = RandomForestRegressor(n_estimators=n_estimators,max_depth=max_depth)
lr.fit(train_x, train_y)
predicted_qualities = lr.predict(test_x)
(rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

print("RandomForest Model (n_estimators=%f, max_depth=%f):" % (n_estimators, max_depth))
print("  RMSE: %s" % rmse)
print("  MAE: %s" % mae)
print("  R2: %s" % r2)

mlflow.log_param("n_estimators", n_estimators)
mlflow.log_param("max_depth", max_depth)
mlflow.log_metric("rmse", rmse)
mlflow.log_metric("r2", r2)
mlflow.log_metric("mae", mae)
mlflow.sklearn.log_model(lr, "model")

RandomForest Model (n_estimators=200.000000, max_depth=10.000000):
  RMSE: 0.009057421756148985
  MAE: 0.006710660830377815
  R2: 0.37834992926304445


## Model Serving

In [19]:
import mlflow.pyfunc

In [20]:
model_predict=mlflow.pyfunc.load_model(model_uri="s3://mlflow-models/fc3fa99044a94af2a35f66de6acae2b5/artifacts/model") 

In [21]:
model_predict.predict([[80,20]])

array([0.01343399])

## Running the strategy with the model

In [23]:
!python /home/jovyan/work/q_pack/q_run/run_BT.py \
--strat_name=simple_strategy_2 \
--strat_param=ml_serving=True,model_uri=fc3fa99044a94af2a35f66de6acae2b5 \
--ml_log=False \
--mode=backtest \
--tickers=EUR_USD,GBP_USD

s3://mlflow-models/fc3fa99044a94af2a35f66de6acae2b5/artifacts/model
Strategy run finished with Run ID: 5
Profit ... or Loss: -1264.36


## Packaging the model using MLflow (BONUS)
For reproducibility and for sharing

In [55]:
mlflow.projects.run("/home/jovyan/work/BT/mlflow_project",parameters={'n_estimators':200,'max_depth':10})


## Serving the model as a rest API using MLflow (BONUS)
Serrve the model in the mlflow tracking container using

docker exec ekholabs-mlflow /bin/sh -c "mlflow models serve -m /ekholabs-mlflow/mlruns/0/a85ab97a393045afaea2b550a79686e8/artifacts/model --host=0.0.0.0 -p 2349"

Then you can call in the program using curl

In [ ]:
! curl -X POST -H "Content-Type:application/json; format=pandas-split" --data '{"columns":["alcohol", "chlorides", "citric acid", "density", "fixed acidity", "free sulfur dioxide", "pH", "residual sugar", "sulphates", "total sulfur dioxide", "volatile acidity"],"data":[[12.8, 2, 10, 0.98, 1, 45, 2, 1.2, 44, 4, 0.66]]}' http://mlflow-image:2349/invocations

In [22]:
! python /home/jovyan/work/q_pack/q_run/run_BT.py \
--strat_name=simple_strategy_2 \
--mode=backtest \
--tickers=EUR_USD,GBP_USD \
--strat_param=model_uri=b5c9975d81c64bc3b662654007e1c20d,ml_serving=True

s3://mlflow-models/b5c9975d81c64bc3b662654007e1c20d/artifacts/model
Profit ... or Loss: -969.07
